# Detection de motifs


## Suffix Trees

In [317]:
from suffix_trees import STree
import random
import numpy as np
import re
from itertools import product
import operator

In [342]:
nuc = ("A", "C", "G", "T")

pattern_len = 8 #taille de motif
subs_nb = 0 #nb de positions variables dans le motif
sequence_nb = 50 #nb de sequences
sequence_len = 100 #longuer des sequence
min_repetition = 4 #nombre de répétitions

In [343]:
# FONCTIONS DES TMEs PRECEDENTS

def isSimple(pattern):
    for c in pattern:
        if len([s.start() for s in re.finditer(c, pattern)]) >= min_repetition:
               return True
    return False
               
def isSimplePair(pattern):
    for i in range(len(pattern)-2):    
        if pattern[i+1:].find(pattern[i:i+2]) != -1:
            return True
    return False
    
def removeSimplePatterns(patterns):
    validPatterns = []
    for key in patterns:
        if not isSimple(key):
               validPatterns.append(key)
    return validPatterns

def removeSimplePairPatterns(patterns):
    validPatterns = []
    for key in patterns:
        if not isSimplePair(key):
            validPatterns.append(key)
    return validPatterns

def hamdist(v, x):
    return sum(map(lambda a,b :int(a.lower()!=b.lower()) ,v,x))

def reverseComplementary(seq):
    """Renvoie le brin complémentaire d’une séquence."""
    complementary = {'A': 'T', 'C': 'G', 'G': 'C', 'T':'A'}
    return [complementary[n] for n in seq[::-1]]

In [344]:
def generateRandomSequence():
    return "".join(random.choices(["A","T","G","C"],weights =[1,1,1,1],k=sequence_nb))

def generateRandomSequences():
    return [generateRandomSequence() for k in range(0,sequence_len)]

def generatePattern():
    pattern = random.choices(["A","T","G","C"],weights =[1,1,1,1],k=pattern_len)
    while (isSimple(''.join(pattern)) or isSimplePair(''.join(pattern))):
        pattern = random.choices(["A","T","G","C"],weights =[1,1,1,1],k=pattern_len)
    return pattern

def substitutePattern(pattern):
    dico_rand={0:"A",1:"T",2:"G",3:"C"}
    pattern_tmp=list(pattern)
    for i in range(0,subs_nb):
        pattern_tmp[int(random.uniform(0,len(pattern_tmp)))]=dico_rand[int(random.uniform(0,4))]
    return "".join(pattern_tmp)

def implantPattern(seq, pattern):
    place_injection_pattern=int(random.uniform(0,len(seq)))
    return seq[:place_injection_pattern]+pattern+seq[place_injection_pattern:]

In [345]:
#generate and implant variable motifs

initial_sequences = generateRandomSequences()
initial_pattern = generatePattern()



sequences_with_initial_pattern = []
for s in initial_sequences:
    sequences_with_initial_pattern.append(implantPattern(s, ''.join(initial_pattern)))
    
print(''.join(initial_pattern))
print(sequences_with_initial_pattern[0])

AATCCTGA
GCAGTTCGAGATACGCAGCTGTAAGTAACCTCTGTATAATCCTGATTGTCGTCCTCGG


In [346]:
def construct_tree(sequences):
    return STree.STree(sequences)

In [347]:
tree = construct_tree(sequences_with_initial_pattern)

In [348]:
# TEST
assert(len(sequences_with_initial_pattern) == len(tree.find_all(''.join(initial_pattern))))

In [349]:
def generateAllKmers(alphabet):
    validKmers = [''.join(c) for c in product(''.join(alphabet), repeat=pattern_len)]
    return removeSimplePairPatterns(removeSimplePatterns(validKmers))

In [350]:
validK = generateAllKmers(nuc)
print(''.join(initial_pattern) in validK)

True


In [351]:
def exact_match(validK, tree):
    res = dict()
    for kmer in validK:
        res[kmer] = len(tree.find_all(kmer))
    return dict(sorted(res.items(), key=operator.itemgetter(1),reverse=True))

In [352]:
pattern_found = exact_match(validK, tree)
print(list(pattern_found.items())[:10])

[('AATCCTGA', 100), ('GAATCCTG', 31), ('ATCCTGAG', 28), ('ATCCTGAA', 26), ('CAATCCTG', 24), ('TAATCCTG', 23), ('ATCCTGAC', 18), ('GGAATCCT', 11), ('TCCTGAGC', 11), ('TCCTGAAC', 10)]


In [353]:
sequences_with_variant = []
for s in initial_sequences:
    sequences_with_variant.append(implantPattern(s, substitutePattern(initial_pattern)))

In [354]:
tree = construct_tree(sequences_with_variant)
patterns_found = exact_match(validK, tree)
print(list(patterns_found.items())[:10])

[('AATCCTGA', 100), ('ATCCTGAA', 29), ('TAATCCTG', 26), ('GAATCCTG', 24), ('ATCCTGAG', 23), ('CAATCCTG', 23), ('ATCCTGAC', 22), ('TCCTGAAC', 11), ('TCCTGAGG', 10), ('TCCTGAAT', 8)]


In [355]:
def inexact_match(kmer, sequences, tree):
    Ns = subs_nb+1
    Ls = int(pattern_len/Ns)
    allCandidates = []
    for i in range(Ns):
        seed = kmer[i*Ls:i*Ls+Ls]
        candidateIndex = tree.find_all(seed)
        for index in candidateIndex:
            candidateText = sequences[index-i*Ls: index+pattern_len-i*Ls]
            if len(candidateText) == pattern_len and hamdist(kmer, candidateText) <= subs_nb:
                allCandidates.append(candidateText)
    return allCandidates

In [1]:
concatS = ''.join(sequences_with_variant)
maxlen = 0
for kmer in validK:
    f = inexact_match(kmer, concatS, tree)
    if len(f) >= maxlen:
        maxlen = len(f)
        maxKmer = kmer
        maxKmerVariant=f
        print(maxKmer, len(f))

NameError: name 'sequences_with_variant' is not defined

In [357]:
# Test avec motif initial TTAGTGCA

Motif LOGO: ![](./logo_motif.png)